In [1]:
import matplotlib 
import matplotlib.pyplot as plt

In [2]:
import example_deeper
import certa.metrics

In [3]:
n_tuples = 2
n_triangles = [8, 10]
#with higher values makes the thresholds (theta_min, theta_max) stricter
threshold_tuning = -2

tests = {
    "cosine" : example_deeper.run(certa.metrics.get_cosine, n_tuples, n_triangles),
    "euclidean" : example_deeper.run(certa.metrics.euclidean_distance, n_tuples, n_triangles),
    "manhattan" : example_deeper.run(certa.metrics.manhattan_distance, n_tuples, n_triangles),
    "mahalanobis" : example_deeper.run(certa.metrics.mahalanobis_distance, n_tuples, n_triangles),
    "jaccard" : example_deeper.run(certa.metrics.jaccard_similarity, n_tuples, n_triangles),
}

In [5]:
tests_results = {}
for test_name, test in tests.items():
    print("Starting test for:", test_name)
    results = []
    for _, _, mean, impact in test:
        result = mean, impact
        print(result)
        results.append(result)
    tests_results[test_name] = results

Starting test for: cosine
Starting test for: euclidean
Model: "functional_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Tupla_A (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
Tupla_B (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
Embedding_lookup (Embedding)    (None, None, 50)     20000100    Tupla_A[0][0]                    
                                                                 Tupla_B[0][0]                    
Total params: 20,000,100
Trainable params: 0
Non-trainable params: 20,000,100
_____________________________________________________

100%|████████████████████████████████████████████████████████████████| 2/2 [00:02<00:00,  1.09s/it]
0it [00:00, ?it/s]


nt: 8 
# of unsuccessful pairs: 0 / 2
(-0.032827749848365784, 0.0)


100%|████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00,  8.09it/s]
0it [00:00, ?it/s]


nt: 10 
# of unsuccessful pairs: 0 / 2
(-0.029980788628260296, 0.0)
Starting test for: manhattan
Model: "functional_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Tupla_A (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
Tupla_B (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
Embedding_lookup (Embedding)    (None, None, 50)     20000100    Tupla_A[0][0]                    
                                                                 Tupla_B[0][0]                    
Total params: 20,000,100
Trainable params: 0
Non-trainable params: 20,000,100
___________

...

nt: 10 
# of unsuccessful pairs: 0 / 2
(-0.045353834118161886, 0.0)
Starting test for: jaccard
Model: "functional_9"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Tupla_A (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
Tupla_B (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
Embedding_lookup (Embedding)    (None, None, 50)     20000100    Tupla_A[0][0]                    
                                                                 Tupla_B[0][0]                    
Total params: 20,000,100
Trainable params: 0
Non-trainable params: 20,000,100
_____________

100%|████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00,  6.88it/s]


nt: 8 
# of unsuccessful pairs: 0 / 2
(0.011271201074123383, 0.0)


100%|████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00,  6.65it/s]


nt: 10 
# of unsuccessful pairs: 0 / 2
(0.0009252279996871948, 0.0)


In [6]:
tests_results

{'cosine': [],
 'euclidean': [(-0.032827749848365784, 0.0), (-0.029980788628260296, 0.0)],
 'manhattan': [(0.011271201074123383, 0.0), (0.0009252279996871948, 0.0)],
 'mahalanobis': [(0.029054418206214905, 0.0), (-0.045353834118161886, 0.0)],
 'jaccard': [(0.011271201074123383, 0.0), (0.0009252279996871948, 0.0)]}

In [7]:
#utility functions


to_feature_index = {
    "mean_drop": 0, 
    "impact": 1
}

'''
extracts single feature from the results of a single metric test based on the name

example: 

results = [(1, 2), (0, 50)]
feature_name = "mean_drop"

output = [1, 0]
'''
def get_feature(results, feature_name):
    feature_name = feature_name.lower()
    feature_index = to_feature_index[feature_name]
    feature_results = []
    for result in results:
        feature_results.append(result[feature_index])
    return feature_results


def get_mean_drops(results):
    return get_feature(results, "mean_drop")
    
def get_impacts(results):
    return get_feature(results, "impact")


def plot_results(mean_metrics_results, title): 
    plt.bar(range(len(mean_metrics_results)), list(mean_metrics_results.values()), align='center')
    plt.xticks(range(len(mean_metrics_results)), list(mean_metrics_results.keys()))
    plt.title(title)
    plt.show()
'''
extracts single feature from the results of all the metric tests

example: 

all_results = {
    "cosine": [(20, 10), (-15, 0)],
    "euclidean": [(77, 12), (42, -13)],
}

feature_name = "mean_drop"

output = {
    "cosine": [20, -15],
    "euclidean": [77, 42],
}
'''
def extract_feature(all_results, feature_name):
    all_results_copy = all_results.copy()
    for test_name, results_raw in all_results_copy.items():
        results_filtered = get_feature(results_raw, feature_name)
        all_results_copy[test_name] = results_filtered
    return all_results_copy


def mean_feature(feature_results):
    import statistics
    mean_results = {}
    for test_name, results in feature_results.items():
        mean_results[test_name] = statistics.mean(results)
    return mean_results

In [8]:
extract_feature(tests_results, "mean_drop")

{'cosine': [],
 'euclidean': [-0.032827749848365784, -0.029980788628260296],
 'manhattan': [0.011271201074123383, 0.0009252279996871948],
 'mahalanobis': [0.029054418206214905, -0.045353834118161886],
 'jaccard': [0.011271201074123383, 0.0009252279996871948]}

In [9]:
extract_feature(tests_results, "impact")

{'cosine': [],
 'euclidean': [0.0, 0.0],
 'manhattan': [0.0, 0.0],
 'mahalanobis': [0.0, 0.0],
 'jaccard': [0.0, 0.0]}

In [10]:
drops = extract_feature(tests_results, "mean_drop")
mean_drops = mean_feature(drops)
mean_drops

StatisticsError: mean requires at least one data point

In [11]:
impacts = extract_feature(tests_results, "impact")
mean_impacts = mean_feature(impacts)
mean_impacts

StatisticsError: mean requires at least one data point

In [ ]:
plot_results(mean_drops, "Mean drop")

In [ ]:
plot_results(mean_impacts, "Mean impacts")